# Lesson 18: 

## SGD 

* First part of lesson goes through SGD, Momentum, RMSProp, Adam and Adam with annealing using a spreadsheet (graddesc.xlsm). 

* Adam works so much better it is quite suprising to me. After one or two runs you have to lower the learnign rate to keep it from bouncing around.

* Adam with annealing attempts to reduce the learning rate as the training progresses.
    * Keeps track of lowest squared gradient.
    *  If the current squared gradient is more than 1/2 the lowest, then it reduces the learning rate by a factor of 4.
    * This was implemented in VBA code in the spreadsheet. Works quite well! 


## Accelerated SGD 

* Notebook #12, same as last time but now up to the Schedulers section

In [1]:
import torch
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl,numpy as np,matplotlib.pyplot as plt
import fastcore.all as fc
from collections.abc import Mapping
from pathlib import Path
from operator import attrgetter,itemgetter
from functools import partial
from copy import copy
from contextlib import contextmanager

import torchvision.transforms.functional as TF,torch.nn.functional as F
from torch import tensor,nn,optim
from torch.utils.data import DataLoader,default_collate
from torch.nn import init
from torch.optim import lr_scheduler
from torcheval.metrics import MulticlassAccuracy
from datasets import load_dataset,load_dataset_builder

from minai.datasets import *
from minai.conv import *
from minai.learner import *
from minai.activations import *
from minai.init import *

from fastcore.test import test_close

torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)

import logging
logging.disable(logging.WARNING)

set_seed(42)

In [2]:
xl,yl = 'image','label'
name = "fashion_mnist"
dsd = load_dataset(name)

bs = 1024
xmean,xstd = 0.28, 0.35

@inplace
def transformi(b): b[xl] = [(TF.to_tensor(o)-xmean)/xstd for o in b[xl]]

tds = dsd.with_transform(transformi)
dls = DataLoaders.from_dd(tds, bs, num_workers=4)

metrics = MetricsCB(accuracy=MulticlassAccuracy())
astats = ActivationStats(fc.risinstance(GeneralRelu))
cbs = [DeviceCB(), metrics, ProgressCB(plot=True), astats]
act_gr = partial(GeneralRelu, leak=0.1, sub=0.4)
iw = partial(init_weights, leaky=0.1)
lrf_cbs = [DeviceCB(), LRFinderCB()]

Jeremy makes the point that we already know how to modify the learning rate, we did this with the learning rate finder. So we are not going to reimplement the schedulers from scratch, instead use the predified schedulers in PyTorch.

In [7]:
# lets see whats available, looking only ot stuff with CamelCase
' '.join([o for o in dir(lr_scheduler) if o[0].isupper() and o[1].islower()])

'ChainedScheduler ConstantLR CosineAnnealingLR CosineAnnealingWarmRestarts Counter CyclicLR ExponentialLR LambdaLR LinearLR MultiStepLR MultiplicativeLR OneCycleLR Optimizer PolynomialLR ReduceLROnPlateau SequentialLR StepLR'

Let's experiment with Consine Annealing..  but first lets see whats in the pytorch optimizer so we can figure out how to get these schedulers working

In [ ]:
# note: The devicecb is required. This seems like a bug in our code. We should default to
# CPU or something.
learn = TrainLearner(get_model(), dls, F.cross_entropy, lr=0.01, cbs = [DeviceCB(), SingleBatchCB()])
learn.fit(1) # need to run at least one epoch to get the optimizer
learn.opt

# at 27 minutes only

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)